In [1]:
import numpy as np

Bijective string transformation
===============================

Take a (binary) word $w\in\{0,1\}^*$, with $w[i]$ meaning the $i$th letter of $w$, starting from $1$. The function **`transform`** implements a bijective string transform $f:\{0,1\}^*\rightarrow \{0,1\}^*$, computed as follows:
* Let $v_1=w$
* Compute $v_{i+1}$ from $v_i$ by taking the `XOR` of pairs of consecutive letters of $v_i$, that is, $|v_{i+1}| = |v_i|-1$, such that $v_{i+1}[j]=v_i[j] \oplus v_i[j+1]$, for all $j\in\{1,|v_i|-1\}$. 
* The last such word, $v_{|w|}$, will have one letter
* The value of $f(w)$ is given by concatenating the first letters of all $v_i$, that is,
$$f(w) = v_1[1]v_2[1]\cdots v_{|w|}[1].$$

$f$ has the following properties, which hold for all $w\in \{0,1\}^*$:
1. **Length preserving:** $|w| = |f(w)|$.
2. **Idempotent:** $w=f(f(w))$, in other form, $w = ff.w$ or $w=f^2.w$.
3. **Commutes(?) with reversal:** $f(f(w)^R) = (f(w^R))^R$, in other form, $fRf.w = RfR.w$.
4. **From 3. we get:** $fRfRfR.w = w$, in other form, $(fR)^3 = 1$.

In [2]:
def transform(word, row=0, full=False):
    size = len(word)
    result = np.zeros((size,size), dtype = np.int)
    if isinstance(word, str):
        start = np.fromiter(word, dtype = np.int)
    else:
        start = word
    result[0] = start
    for i in np.arange(1, size):
        result[i, :size-i] = (result[i-1, : size-i] + result[i-1, 1: size-i+1])%2
    if full:
        return result
    elif row > 0 and row < size:
        return result[row]
    else:
        return result[:,0]

## A printing method for the steps of computation of the transform. Row $i$ corresponds to $v_i$ ##

In [3]:
def triprint(matrix):
    size = matrix.shape[0]
    for i in np.arange(size):
        print((' '*i+' '.join(map(str, matrix[i,:size-i]))).replace('0','.'))

## We can define order $k$ complements of a binary string, based on the transform. There are at least two straightforward choices, which both agree with order 1 complement being the usual notion of negating individual bits. ##

One choice is to negate the $k$-th bit of the transform and apply the transform to the result. The other is to negate all bits up to position $k$ in the transform, and apply the transform again. In symbols, if $C_k(v)=v[1]\cdots v[k-1]\cdot \overline{v[k]}\cdot v[k+1]\cdots v[|v|]$, then the first type of complementation of order $k$, amounts to $X_k(w) = fC_kf.w$, whereas the second type of complementation of order $k$ is defined by $Y_k(w) = fC_kC_{k-1}\cdots C_1f.w$. Both types of complements of order $k$, for any $k$, are involutions, i.e., equal to their own inverses.

In [4]:
def complement1(word, order):
    tr = transform(word)
    tr[order] = 1 - tr[order]
    return transform(tr)

In [5]:
complement1('10110',2), complement1(complement1('10110',2),2)

(array([1, 0, 0, 0, 0]), array([1, 0, 1, 1, 0]))

In [6]:
def complement2(word, order):
    tr = transform(word)
    tr[:order+1] = 1 - tr[:order+1]
    return transform(tr)

In [7]:
complement2('10110',2), complement2(complement2('10110',2),2)

(array([0, 0, 1, 0, 1]), array([1, 0, 1, 1, 0]))

In [8]:
word = '01010101'
for i in range(len(word)):
    print(complement1(word, i), '               ***             ', complement2(word, i))

[1 0 1 0 1 0 1 0]                ***              [1 0 1 0 1 0 1 0]
[0 0 0 0 0 0 0 0]                ***              [1 1 1 1 1 1 1 1]
[0 1 1 0 0 1 1 0]                ***              [1 1 0 0 1 1 0 0]
[0 1 0 0 0 1 0 0]                ***              [1 1 0 1 1 1 0 1]
[0 1 0 1 1 0 1 0]                ***              [1 1 0 1 0 0 1 0]
[0 1 0 1 0 0 0 0]                ***              [1 1 0 1 0 1 1 1]
[0 1 0 1 0 1 1 0]                ***              [1 1 0 1 0 1 0 0]
[0 1 0 1 0 1 0 0]                ***              [1 1 0 1 0 1 0 1]


In [9]:
"1010011".replace('1','.')

'.0.00..'

In [10]:
triprint(transform('101001000101001000',full=True))

1 . 1 . . 1 . . . 1 . 1 . . 1 . . .
 1 1 1 . 1 1 . . 1 1 1 1 . 1 1 . .
  . . 1 1 . 1 . 1 . . . 1 1 . 1 .
   . 1 . 1 1 1 1 1 . . 1 . 1 1 1
    1 1 1 . . . . 1 . 1 1 1 . .
     . . 1 . . . 1 1 1 . . 1 .
      . 1 1 . . 1 . . 1 . 1 1
       1 . 1 . 1 1 . 1 1 1 .
        1 1 1 1 . 1 1 . . 1
         . . . 1 1 . 1 . 1
          . . 1 . 1 1 1 1
           . 1 1 1 . . .
            1 . . 1 . .
             1 . 1 1 .
              1 1 . 1
               . 1 1
                1 .
                 1


In [11]:
triprint(transform('0110100110010110',full=True))

. 1 1 . 1 . . 1 1 . . 1 . 1 1 .
 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1
  1 1 . . 1 1 1 1 1 1 . . 1 1
   . 1 . 1 . . . . . 1 . 1 .
    1 1 1 1 . . . . 1 1 1 1
     . . . 1 . . . 1 . . .
      . . 1 1 . . 1 1 . .
       . 1 . 1 . 1 . 1 .
        1 1 1 1 1 1 1 1
         . . . . . . .
          . . . . . .
           . . . . .
            . . . .
             . . .
              . .
               .


## Helper function `fib`(n) returns $h^n(0)$, where $h$ is the morphism generating the Fibonacci word, that is, $h(0)=01$ and $h(1)=0$.

In [12]:
def fib(n):
    if n<=0:
        return "0"
    elif n==1:
        return "01"
    else:
        return fib(n-1)+fib(n-2)

In [13]:
fib(10)

'010010100100101001010010010100100101001010010010100101001001010010010100101001001010010010100101001001010010100100101001001010010100100101001010'

## Helper function `TM`(n) returns $h^n(0)$, where $h$ is the morphism generating the Thue-Morse word, that is, $h(0)=01$ and $h(1)=10$.

In [14]:
def TM(n):
    if n==0:
        return '0'
    else:
        result = TM(n-1)
        result = result.replace('0', 'x')
        result = result.replace('1', '0')
        result = result.replace('x', '1')
        return TM(n-1)+result

In [15]:
TM(4)

'0110100110010110'

## The complement operations seem to commute for both types and all orders $k$, that is, for all words $w$ and orders $1\leq k_i,k_j\leq |w|$ we have 

* $$X_{k_1}(Y_{k_2}(w))=Y_{k_2}(X_{k_1}(w)).$$
* $$X_{k_1}(X_{k_2}(w))=X_{k_2}(X_{k_1}(w)).$$
* $$Y_{k_1}(Y_{k_2}(w))=Y_{k_2}(Y_{k_1}(w)).$$

In [16]:
complement2('1100',2), complement2('101001101',2)

(array([0, 1, 0, 1]), array([0, 0, 1, 1, 1, 1, 1, 1, 0]))

In [17]:
complement1(complement1(complement1('0111001101111010',2),5),3)

array([0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1])

In [18]:
complement1(complement2('0111001101111010',2),5)==complement2(complement1('0111001101111010',5),2)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [19]:
complement1(complement1(complement1(complement1(complement1(complement1('0111001101111010',2),5),3),2),5),3)

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0])

In [20]:
complement1(complement1(complement1(complement1(complement1('0111001101111010',2),5),1),2),5)

array([0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1])

## The complementation transform does not preserve periodicity, e.g., below, 
## $X_8((10110)^5) =$ 10110 10010 10110 00110 10100.

In [21]:
complement1(5*'10110',7)

array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0])

## Prefix normal form word

In [22]:
triprint(transform('1110100110100101100101',full=True))

1 1 1 . 1 . . 1 1 . 1 . . 1 . 1 1 . . 1 . 1
 . . 1 1 1 . 1 . 1 1 1 . 1 1 1 . 1 . 1 1 1
  . 1 . . 1 1 1 1 . . 1 1 . . 1 1 1 1 . .
   1 1 . 1 . . . 1 . 1 . 1 . 1 . . . 1 .
    . 1 1 1 . . 1 1 1 1 1 1 1 1 . . 1 1
     1 . . 1 . 1 . . . . . . . 1 . 1 .
      1 . 1 1 1 1 . . . . . . 1 1 1 1
       1 1 . . . 1 . . . . . 1 . . .
        . 1 . . 1 1 . . . . 1 1 . .
         1 1 . 1 . 1 . . . 1 . 1 .
          . 1 1 1 1 1 . . 1 1 1 1
           1 . . . . 1 . 1 . . .
            1 . . . 1 1 1 1 . .
             1 . . 1 . . . 1 .
              1 . 1 1 . . 1 1
               1 1 . 1 . 1 .
                . 1 1 1 1 1
                 1 . . . .
                  1 . . .
                   1 . .
                    1 .
                     1


In [23]:
triprint(transform(complement2(fib(6),15), full=True))

1 1 . . 1 . 1 . . 1 . . 1 . 1 . 1 1 . 1 .
 . 1 . 1 1 1 1 . 1 1 . 1 1 1 1 1 . 1 1 1
  1 1 1 . . . 1 1 . 1 1 . . . . 1 1 . .
   . . 1 . . 1 . 1 1 . 1 . . . 1 . 1 .
    . 1 1 . 1 1 1 . 1 1 1 . . 1 1 1 1
     1 . 1 1 . . 1 1 . . 1 . 1 . . .
      1 1 . 1 . 1 . 1 . 1 1 1 1 . .
       . 1 1 1 1 1 1 1 1 . . . 1 .
        1 . . . . . . . 1 . . 1 1
         1 . . . . . . 1 1 . 1 .
          1 . . . . . 1 . 1 1 1
           1 . . . . 1 1 1 . .
            1 . . . 1 . . 1 .
             1 . . 1 1 . 1 1
              1 . 1 . 1 1 .
               1 1 1 1 . 1
                . . . 1 1
                 . . 1 .
                  . 1 1
                   1 .
                    1


In [24]:
triprint(transform('1001101110', full=True))

1 . . 1 1 . 1 1 1 .
 1 . 1 . 1 1 . . 1
  1 1 1 1 . 1 . 1
   . . . 1 1 1 1
    . . 1 . . .
     . 1 1 . .
      1 . 1 .
       1 1 1
        . .
         .


In [25]:
triprint(transform('1001101110'+'1110001100', full=True))

1 . . 1 1 . 1 1 1 . 1 1 1 . . . 1 1 . .
 1 . 1 . 1 1 . . 1 1 . . 1 . . 1 . 1 .
  1 1 1 1 . 1 . 1 . 1 . 1 1 . 1 1 1 1
   . . . 1 1 1 1 1 1 1 1 . 1 1 . . .
    . . 1 . . . . . . . 1 1 . 1 . .
     . 1 1 . . . . . . 1 . 1 1 1 .
      1 . 1 . . . . . 1 1 1 . . 1
       1 1 1 . . . . 1 . . 1 . 1
        . . 1 . . . 1 1 . 1 1 1
         . 1 1 . . 1 . 1 1 . .
          1 . 1 . 1 1 1 . 1 .
           1 1 1 1 . . 1 1 1
            . . . 1 . 1 . .
             . . 1 1 1 1 .
              . 1 . . . 1
               1 1 . . 1
                . 1 . 1
                 1 1 1
                  . .
                   .


In [26]:
triprint(transform(TM(7), full=True))

. 1 1 . 1 . . 1 1 . . 1 . 1 1 . 1 . . 1 . 1 1 . . 1 1 . 1 . . 1 1 . . 1 . 1 1 . . 1 1 . 1 . . 1 . 1 1 . 1 . . 1 1 . . 1 . 1 1 . 1 . . 1 . 1 1 . . 1 1 . 1 . . 1 . 1 1 . 1 . . 1 1 . . 1 . 1 1 . . 1 1 . 1 . . 1 1 . . 1 . 1 1 . 1 . . 1 . 1 1 . . 1 1 . 1 . . 1
 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1 . 1 . 1 1 1 . 1
  1 1 . . 1 1 1 1 1 1 . . 1 1 . . 1 1 . . 1 1 1 1 1 1 . . 1 1 1 1 1 1 . . 1 1 1 1 1 1 . . 1 1 . . 1 1 . . 1 1 1 1 1 1 . . 1 1 . . 1 1 . . 1 1 1 1 1 1 . . 1 1 . . 1 1 . . 1 1 1 1 1 1 . . 1 1 1 1 1 1 . . 1 1 1 1 1 1 . . 1 1 . . 1 1 . . 1 1 1 1 1 1 . . 1 1
   . 1 . 1 . . . . . 1 . 1 . 1 . 1 . 1 . 1 . . . . . 1 . 1 . . . . . 1 . 1 . . . . . 1 . 1 . 1 . 1 . 1 . 1 . . . . . 1 . 1 . 1 . 1 . 1 . 1 . . . . . 1 . 1 . 1 . 1 . 1 . 1 . . . . . 1 . 1 . . . . . 1 . 1 . . . . . 1 . 1 . 1 . 1 . 1 . 1 

In [27]:
triprint(transform(fib(9),full=True))

. 1 . . 1 . 1 . . 1 . . 1 . 1 . . 1 . 1 . . 1 . . 1 . 1 . . 1 . . 1 . 1 . . 1 . 1 . . 1 . . 1 . 1 . . 1 . 1 . . 1 . . 1 . 1 . . 1 . . 1 . 1 . . 1 . 1 . . 1 . . 1 . 1 . . 1 . . 1
 1 1 . 1 1 1 1 . 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 . 1 1 1 1 . 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 . 1 1 1 1 . 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 . 1 1 1 1 . 1 1 . 1
  . 1 1 . . . 1 1 . 1 1 . . . 1 1 . . . 1 1 . 1 1 . . . 1 1 . 1 1 . . . 1 1 . . . 1 1 . 1 1 . . . 1 1 . . . 1 1 . 1 1 . . . 1 1 . 1 1 . . . 1 1 . . . 1 1 . 1 1 . . . 1 1 . 1 1
   1 . 1 . . 1 . 1 1 . 1 . . 1 . 1 . . 1 . 1 1 . 1 . . 1 . 1 1 . 1 . . 1 . 1 . . 1 . 1 1 . 1 . . 1 . 1 . . 1 . 1 1 . 1 . . 1 . 1 1 . 1 . . 1 . 1 . . 1 . 1 1 . 1 . . 1 . 1 1 .
    1 1 1 . 1 1 1 . 1 1 1 . 1 1 1 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1 1 1 1 . 1 1 1 . 1 1 1 . 1 1 1 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1 1 1 1 . 1 1 1 . 1 1 1 . 1 1 1 . 1
     . . 1 1 . . 1 1 . . 1 1 . . . 1 1 . . 1 1 . . 1 1 . . 1 1 . . 1 1 . . . 1 1 . . 1 1 . . 1 1 . . . 1 1 . . 1 1 . . 1

In [28]:
triprint(transform(10*'01101', full=True))

. 1 1 . 1 . 1 1 . 1 . 1 1 . 1 . 1 1 . 1 . 1 1 . 1 . 1 1 . 1 . 1 1 . 1 . 1 1 . 1 . 1 1 . 1 . 1 1 . 1
 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1
  1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 .
   . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1
    1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1
     . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . .
      1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 .
       1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1 1
        . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . . 1 1 . . .
         . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . . 1 . 1 . .
          1 1 1 1 . 1 1 1 1 . 1 1 1 1 . 1 1 1

In [29]:
triprint(transform(30*'0'+'1'+30*'0', full=True))

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1 1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
  . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1 . 1 . . . . . . . . . . . . . . . . . . . . . . . . . . . .
   . . . . . . . . . . . . . . . . . . . . . . . . . . . 1 1 1 1 . . . . . . . . . . . . . . . . . . . . . . . . . . .
    . . . . . . . . . . . . . . . . . . . . . . . . . . 1 . . . 1 . . . . . . . . . . . . . . . . . . . . . . . . . .
     . . . . . . . . . . . . . . . . . . . . . . . . . 1 1 . . 1 1 . . . . . . . . . . . . . . . . . . . . . . . . .
      . . . . . . . . . . . . . . . . . . . . . . . . 1 . 1 . 1 . 1 . . . . . . . . . . . . . . . . . . . . . . . .
       . . . . . . . . . . . . . . . . . . . . . . . 1 1 1 1 1 1 1 1 . . . . . . . . . . . . . . . . . . . . . . .
        . . . . . . . . . . . . . . . . . . . . . . 

In [30]:
triprint(transform('00001101', full=True))

. . . . 1 1 . 1
 . . . 1 . 1 1
  . . 1 1 1 .
   . 1 . . 1
    1 1 . 1
     . 1 1
      1 .
       1


In [40]:
complement2('01000',2)

array([1, 1, 0, 1, 1])